In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# IMPORTS 

from pathlib import Path 
import os 

import pandas as pd
import numpy as np

In [ ]:
# PATHS 
DATA_FOLDER = Path(*Path().absolute().parts[:-1]) / 'data' # directory containing the notebook

Load data


In [ ]:
import json 

json_file_path_train = DATA_FOLDER / 'Twibot-20/train.json'
json_file_path_val = DATA_FOLDER / 'Twibot-20/dev.json'
json_file_path_test = DATA_FOLDER / 'Twibot-20/test.json'

with open(json_file_path_train, 'r') as tr:
     contents = json.loads(tr.read())
     train_df = pd.json_normalize(contents)
     train_df['split'] = 'train'

with open(json_file_path_val, 'r') as vl:
     contents = json.loads(vl.read())
     val_df = pd.json_normalize(contents) 
     val_df['split'] = 'val'

with open(json_file_path_test, 'r') as ts:
     contents = json.loads(ts.read())
     test_df = pd.json_normalize(contents) 
     test_df['split'] = 'test'

df = pd.concat([train_df,val_df,test_df],ignore_index=True) # merge three datasets
df.dropna(subset=['tweet'], inplace=True)  # remove rows withot any tweet 
df.set_index(keys='ID',inplace=True) # reset index

# split dataframe in two : tweet and account data 
tweets_df = df[['tweet','label','split']].reset_index()
tweets_df = tweets_df.explode('tweet').reset_index(drop=True)
tweets_df.rename(columns={"ID": "account_id"}, inplace=True)

account_df = df.drop('tweet',axis=1).reset_index()
account_df.rename(columns={"ID": "account_id"}, inplace=True)


DATA PROCESSING AND CLEANING  

In [ ]:
import emoji
from nltk.tokenize import TweetTokenizer

from ttp import ttp 
parser = ttp.Parser(include_spans=True)

from emot.core import emot
emot_obj = emot()

tk = TweetTokenizer(reduce_len=True,preserve_case=False)

CASHTAG = "(?<!\S)\$[A-Z]+(?:\.[A-Z]+)?(?!\S)"   # to check  (?:\.[A-Z]+)?
EMAIL = r"""[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]"""
MONEY = "[$£][0-9]+(?:[.,]\d+)?[k+B]?|[0-9]+(?:[.,]\d+)?[k+B]?[$£]"  
NUMBERS = r"""(?<!\S)(?:[+\-]?\d+(?:%|(?:[,/.:-]\d+[+\-]?)?))"""   # r"""(?:[+\-]?\d+[,/.:-]\d+[+\-]?)"""   
HASHTAG = r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
HANDLES = r"""(?:@[\w_]+)""" 


TO_REPLACE = [CASHTAG, EMAIL, MONEY, NUMBERS, HASHTAG, HANDLES]
REPLACE_WITH = [' stock ',' email ',' money ',' number ',' hashtag ',' username ']

tweets_df['processed_tweet'] = tweets_df['tweet'].replace(TO_REPLACE,REPLACE_WITH,regex=True,inplace=False)

URL_KEYWORDS = ['https', 'http', '.com']

def further_process(sentence: str):
        #replace urls 
        result = parser.parse(sentence, html=False)
        urls = dict(result.urls).keys()
        for url in urls:
                sentence = sentence.replace(url,' url ')
        
        #replace emoticons 
        emoticons = emot_obj.emoticons(sentence)
        for emoticon in emoticons['value']:
                sentence = sentence.replace(emoticon,' emoticon ')
        
        #replace emoji
        sentence = emoji.replace_emoji(sentence,' emoji ')

        #tokenize
        sentence = tk.tokenize(sentence)

        #replace residual urls 
        sentence = [' url ' if any(k in word for k in URL_KEYWORDS) else word for word in sentence]
                
        return sentence
        

tweets_df['processed_tweet'] = tweets_df['processed_tweet'].apply(further_process)

tweets_df.to_pickle(DATA_FOLDER / 'processed_dataset.pkl')   #save to file 

DOWNLOAD TWITTER GLOVE EMBEDDINGS

In [14]:
import gensim
import gensim.downloader as gloader
from gensim.models import KeyedVectors

glove_model_cached_path = DATA_FOLDER / 'glove_vectors.txt'
glove_model_download_path = 'glove-twitter-200'
force_download = False  # to download glove model even if the vectors model has been already stored. Mainly for testing purposes

if os.path.exists(glove_model_cached_path) and not force_download: 
    print('found cached glove vectors in data folder, retrieving the file...')
    emb_model = KeyedVectors.load_word2vec_format(glove_model_cached_path, binary=True)
    print('vectors loaded')

else:
    print('downloading glove embeddings...')        
    emb_model = gloader.load(glove_model_download_path)

    print('saving glove embeddings to file')  
    emb_model.save_word2vec_format(glove_model_cached_path, binary=True)

found cached glove vectors in data folder, retrieving the file...
vectors loaded


CUSTOM DATASET 